In [4]:
import tarfile
import re
from pathlib import Path

import pandas as pd

In [46]:
source_folder = Path(snakemake.config["raw_sim_folder"])/snakemake.wildcards["sim"]
re_numbers = r"[0-9]+"

In [5]:
with tarfile.open(snakemake.output["data"], 'w') as dest:
    for data_tar in source_folder.glob("data/*.tar"):
        tar = tarfile.open(data_tar, 'r')
        members = tar.getmembers()
        for member in members:
            if member.isdir():
                continue
            fileobj = tar.extractfile(member)
            member.name = Path(member.name).name
            dest.addfile(member, fileobj)
        tar.close()

In [38]:
if snakemake.params["combine_features"]:
    features_file = Path(snakemake.output["data"]).parent / "features.tar.gz"
    
    with tarfile.open(features_file, 'w:gz') as dest:
        for data_tar in source_folder.glob("features/*.tar.gz"):
            tar = tarfile.open(data_tar, 'r:gz')
            members = tar.getmembers()
            for member in members:
                if member.isdir():
                    continue
                fileobj = tar.extractfile(member)
                member.name = re.search(re_numbers, data_tar.stem).group() + "-" + Path(member.name).name
                dest.addfile(member, fileobj)
            tar.close()

In [43]:
pd.read_table?

In [70]:
param_dfs = []

for f in source_folder.glob("parameters/*.tsv"):
    batch_id = re.search(re_numbers, f.stem).group()
    df = pd.read_table(f, dtype={"data_id": str})
    df = df.assign(data_id = pd.Series([str(batch_id) for _ in range(len(df))]).str.cat(df.data_id, sep='-'))
    param_dfs.append(df)
    
params = pd.concat(param_dfs).reset_index(drop=True)

In [74]:
params.to_csv(snakemake.output["parameters"], index=False, sep="\t")